# Polars

In [2]:
import polars as pl

In [13]:
# Green
green_schema = pl.read_parquet("green_tripdata_2025-01.parquet", columns=['total_amount', 'trip_distance', 'tip_amount', 'tolls_amount'])
green_avg = green_schema['total_amount'].mean()
print(f"Promedio de Taxis Green: ${green_avg:.2f}")
# Yellow
yellow_schema = pl.read_parquet('yellow_tripdata_2025-01.parquet', columns=['total_amount', 'trip_distance', 'tip_amount', 'tolls_amount'])
yellow_avg = yellow_schema['total_amount'].mean()
print(f"Promedio de Taxis Yellow: ${yellow_avg:.2f}")

# fhvhv  scan_parquet no carga todo el archivo en memoria
fhvhv_lazy = pl.scan_parquet('fhvhv_tripdata_2025-01.parquet')
fhvhv_income = fhvhv_lazy.select(
    (
        pl.col('base_passenger_fare') +
        pl.col('tolls') +
        pl.col('bcf') +
        pl.col('sales_tax') +
        pl.col('congestion_surcharge') +
        pl.col('airport_fee') +
        pl.col('tips')
    ).alias('total_income')
)

# Calcular el promedio (y ejecutar el plan)
fhvhv_avg = fhvhv_income.select(pl.col('total_income').mean()).collect()[0, 0]
print(f"FHVHV avg income: ${fhvhv_avg:.2f}")

#Promedio de taxis
promedio_taxis= sum([green_avg, yellow_avg, fhvhv_avg])/3
print(f"El promedio de los 3 tipos de Taxis es de ${promedio_taxis:.2f}")


Promedio de Taxis Green: $22.63
Promedio de Taxis Yellow: $25.61
FHVHV avg income: $30.24
El promedio de los 3 tipos de Taxis es de $26.16


### Mayor cantidad de distancia

In [22]:
green_distance = green_schema['trip_distance'].sum()
yellow_distance = yellow_schema["trip_distance"].sum()
#fhvhv_distance = fhvhv_lazy.select(pl.col('trip_miles')).sum()

# Mostrar resultados
print(f"Distancia total Green: {green_distance:.2f} millas")
print(f"Distancia total Yellow: {yellow_distance:.2f} millas")

# Leer fhvhv en modo lazy 
fhvhv_lazy = pl.read_parquet("fhvhv_tripdata_2025-01.parquet", columns=["trip_miles"]).lazy()

# Calcular distancia total
fhvhv_total = (
    fhvhv_lazy
    .select(pl.col("trip_miles").sum().alias("total_millas"))
    .collect()
    .item(0, 0)
)

print(f"Distancia total FHVHV: {fhvhv_total:.2f} millas")

# Determinar cuál fue mayor
distancias = {
    "Green": green_distance,
    "Yellow": yellow_distance,
    "FHVHV": fhvhv_total
}

tipo_mayor = max(distancias, key=distancias.get)
print(f"\n El tipo de taxi que recorrió más distancia en enero fue: {tipo_mayor} con {distancias[tipo_mayor]:.2f} millas.")

Distancia total Green: 1040575.25 millas
Distancia total Yellow: 20347886.73 millas
Distancia total FHVHV: 98978656.51 millas

 El tipo de taxi que recorrió más distancia en enero fue: FHVHV con 98978656.51 millas.


### Tipo de Taxi con más propinas

In [25]:
green_tips = green_schema['tip_amount'].sum()
yellow_tips = yellow_schema['tip_amount'].sum()

# Leer fhvhv en modo lazy 
fhvhv_lazy = pl.read_parquet("fhvhv_tripdata_2025-01.parquet", columns=["tips"]).lazy()

# Calcular propinas totales
fhvhv_total = (
    fhvhv_lazy
    .select(pl.col("tips").sum().alias("total_tips"))
    .collect()
    .item(0, 0)
)

print(f"Propinas de Taxis Verdes: {green_tips:.2f} USD")
print(f"Propinas de Taxis Yellow: {yellow_tips:.2f} USD")
print(f"Propinas FHVHV: {fhvhv_total:.2f} USD")

propinas = {
    "Green": green_tips,
    "Yellow": yellow_tips,
    "FHVHV": fhvhv_total
}

tipo_mayor = max(propinas, key=propinas.get)
print(f"\n El tipo de taxi que recibió más propinas en enero fue: {tipo_mayor} con {propinas[tipo_mayor]:.2f} USD.")

Propinas de Taxis Verdes: 119938.32 USD
Propinas de Taxis Yellow: 10286018.35 USD
Propinas FHVHV: 21247787.65 USD

 El tipo de taxi que recibió más propinas en enero fue: FHVHV con 21247787.65 USD.
